# Notes 


In [5]:
%load_ext tclmagic

#  If the prev line doesnt work  install it using the next line in your unix terminal
#  /depot/Python/Python-3.8.0/bin/pip install -U tcl-magic
#  /depot/Python/Python-3.8.0/bin/pip install -U notebook
#  You can configure it to  

The tclmagic extension is already loaded. To reload it, use:
  %reload_ext tclmagic


### Init our libraries and packages

In [6]:
%%tcl 

set SHELLTOOL_LOC "/remote/cad-rep/msip/tools/Shelltools"
lappend auto_path "$SHELLTOOL_LOC/ddr-utils-lay/dev/lib/tcl"
lappend auto_path "/depot/tcl8.6.3/lib"

set RealBin [file dirname [file normalize [info script]] ]
set RealScript [file tail [file normalize [info script]] ]
set PROGRAM_NAME $RealScript
set LOGFILE "[pwd]/log-$PROGRAM_NAME.log"

package require Messaging 1.0
namespace import ::Messaging::*
package require Misc 1.0
namespace import ::Misc::*

package require cmdline
package require try
package require yaml

'0.3.10'

In [29]:
%%tcl
cd /u/alvaro/GitLab/my-personal-projects/experiments/ddr-utils-lay/bin/

## creating a proc that source and join data in a variable

In [35]:
%%tcl
variable scriptDir [file dirname [info script]]
proc invoke_floorplan {scriptDir {floorplan_file "crd_abutment_floorplans.tcl"}} {
    variable $scriptDir
    source [file join $scriptDir $floorplan_file]
}
set scriptDir [invoke_floorplan $scriptDir]
puts $scriptDir


dwc_lpddr5xmphyzcal_top_ew 1 1 0 0 0 0 0 0 dwc_lpddr5xmphy_decapvsh_zcal_ew 2 1 {$x_dwc_lpddr5xmphy_decapvsh_zcal_ew} 0 {$x_dwc_lpddr5xmphyzcal_top_ew} 0 0 0 dwc_lpddr5xmphy_decapvddq_ns 5 1 {$x_dwc_lpddr5xmphy_decapvddq_ns} 0 0 {$y_dwc_lpddr5xmphyzcal_top_ew} 0 0 dwc_lpddr5xmphy_decapvddq_ns 5 1 {$x_dwc_lpddr5xmphy_decapvddq_ns} 0 0 {-$y_dwc_lpddr5xmphy_decapvddq_ns} 0 0


In [10]:
%%tcl
proc add2list {list_name new_value} {
    upvar $list_name lister
    if {[info exists lister]} {
        if {[lsearch -exact $lister $new_value] == -1} {
            lappend lister $new_value
        }
    } else {
        lappend lister $new_value
    }
}

## Check libs for all cells
 to confirm and check that all cells have their libs, otherwise return

In [11]:
%%tcl 
proc check_cells_libs {macros} {
    foreach macro $macros {
      if {[array names libraries -exact $macro] == ""} {
        de::sendMessage "No reference library found for $macro. Exiting." -severity error
        return -level [info level] 1
      }
    }
}

## Get list and found libraries

It should go before the previous points, but it is.

In [13]:
%%tcl
#floorplan was gotten previosuly with the invoke_floorplan
proc get_macros_from_testcases {testcases} {
    foreach testcase $testcases {
        variable floorplans($testcase)
        foreach {macro columns rows d_x d_y x y angle mirror} $floorplans($testcase) {
            lappend macros $macro
        }
    }
    return $macros
}

In [14]:
%%tcl
proc check_lib_layoutview {macros reflibs} {
    set macros [lsort -unique $macros]
    foreach macro $macros {
        foreach lib $reflibs {
            # First check that cell exists in lib, then check if layout view exists.
            if {![db::isEmpty  [dm::getCells $macro -libName $lib]] && ![db::isEmpty [dm::getCellViews layout -cellName $macro -libName $lib]]} {
                variable libraries($macro) $lib
                break
            }
        }
    }
}

## Getting macros sizes

In [20]:
%%tcl 

# Get macro sizes.
proc get_macro_sizes {macros} {
    foreach macro $macros {
        variable libraries($macro)
        set context [de::open [dm::getCellViews layout -cellName $macro -libName $libraries($macro)] -readOnly true -headless true]
        set design [db::getAttr editDesign -of $context]
        set block [oa::getTopBlock $design]
        set boundary [oa::PRBoundaryFind $block]
        #set widths($macro) [lindex [db::getAttr bBox -of $boundary] 1 0]
        variable x_$macro [lindex [db::getAttr bBox -of $boundary] 1 0]
        #set heights($macro) [lindex [db::getAttr bBox -of $boundary] 1 1]
        variable y_$macro [lindex [db::getAttr bBox -of $boundary] 1 1]
        de::close $context
    }
}

In [ ]:
%%tcl

#Generate testcases
proc generate_testcases {testcases destlib} {
    foreach testcase $testcases {
        variable floorplans($testcase)
        set cell [dm::createCell $testcase -libName $destlib]
        set cellView [dm::createCellView layout -cell $cell -viewType maskLayout]
        set context [de::open [dm::getCellViews layout -cellName $testcase -libName $destlib] -headless true]
        set design [db::getAttr editDesign -of $context]

        foreach {macro columns rows dx dy x y angle mirror} $floorplans($testcase) {
            switch -- "$angle $mirror" {
                "0 0"   {set orientation R0}
                "0 1"   {set orientation MX}
                "180 1" {set orientation MY}
                "180 0" {set orientation R180}
            }
            le::createInst -libName $libraries($macro) -cellName $macro -viewName layout -design $design -orient $orientation -origin "[expr $x] [expr $y]" -rows $rows -cols $columns -dx [expr $dx] -dy [expr $dy]
        }
        de::save $context
        de::close $context    
    }
}

In [ ]:
%%tcl

proc launchCRDTestcaseCreation {} {
    variable scriptDir

    # Source floorplans array from crd_abutment_floorplans.tcl file in same directory as script.
    set scriptDir [invoke_floorplan]

    # Get list of libraries.
    set libs [db::createList [db::getAttr name -of [dm::getLibs]]]

    # Generate GUI dialog box.
    set crdDialog       [gi::createDialog crdDialog -title "CRD Testcase Creation" -showHelp 0 -execProc cc_crd_abutment::CRDTestcaseCreationExecProc]
    set destlibInput    [dm::createLibInput destlibInput -parent $crdDialog -label "Destination Library"]
    set testcasesInput  [gi::createListInput testcasesInput -parent $crdDialog -label "Testcases to Generate" -header "Testcases" -items [array names floorplans] -showFilter -selectionModel multiple -viewType checkbox]
    set reflibsInput    [gi::createListInput reflibsInput -parent $crdDialog -label "Reference Libraries" -header "Available Selected" -items $libs -selectionModel multiple -viewType dualList]
    set haloInputs      [gi::createInlineGroup haloInputs -parent $crdDialog -label "Boundary Upsize (Halo) (um)"]
    
    foreach side {bottom left right top} {
      set ${side}HaloInput [gi::createNumberInput ${side}HaloInput -parent $haloInputs -label [string toupper $side 0 0] -valueType float -value 5]
    }
    
    set layersInput [gi::createListInput layersInput -parent $crdDialog -label "Layers for Pin Propagation" -header "Layers" -items "M0 M1 M2 M3 M4 M5 M6 M7 M8 M9 M10 M11 M12 M13 M14 M15 M16 M17 M18 MTOP-1 MTOP RDL" -selectionModel multiple -viewType checkbox]
}

In [ ]:
%%tcl

namespace eval cc_crd_abutment {

    variable scriptDir [file dirname [info script]]

    proc invoke_floorplan {scriptDir {floorplan_file "crd_abutment_floorplans.tcl"}} {
        variable $scriptDir
        return [file join $scriptDir $floorplan_file]
    }

    proc check_cells_libs {macros} {
        foreach macro $macros {
        if {[array names libraries -exact $macro] == ""} {
            de::sendMessage "No reference library found for $macro. Exiting." -severity error
            return -level [info level] 1
        }
        }
    }

    proc get_macros_from_testcases {testcases} {
        foreach testcase $testcases {
            variable floorplans($testcase)
            foreach {macro columns rows d_x d_y x y angle mirror} $floorplans($testcase) {
                lappend macros $macro
            }
        }
        return $macros
    }

    proc check_lib_layoutview {macros reflibs} {
        set macros [lsort -unique $macros]
        foreach macro $macros {
            foreach lib $reflibs {
                # First check that cell exists in lib, then check if layout view exists.
                if {![db::isEmpty  [dm::getCells $macro -libName $lib]] && ![db::isEmpty [dm::getCellViews layout -cellName $macro -libName $lib]]} {
                    variable libraries($macro) $lib
                    break
                }
            }
        }
    }

    proc get_macro_sizes {macros} {
        foreach macro $macros {
            variable libraries($macro)
            set context [de::open [dm::getCellViews layout -cellName $macro -libName $libraries($macro)] -readOnly true -headless true]
            set design [db::getAttr editDesign -of $context]
            set block [oa::getTopBlock $design]
            set boundary [oa::PRBoundaryFind $block]
            #set widths($macro) [lindex [db::getAttr bBox -of $boundary] 1 0]
            variable x_$macro [lindex [db::getAttr bBox -of $boundary] 1 0]
            #set heights($macro) [lindex [db::getAttr bBox -of $boundary] 1 1]
            variable y_$macro [lindex [db::getAttr bBox -of $boundary] 1 1]
            de::close $context
        }
    }

    proc generate_testcases {testcases destlib} {
        foreach testcase $testcases {
            variable floorplans($testcase)
            set cell [dm::createCell $testcase -libName $destlib]
            set cellView [dm::createCellView layout -cell $cell -viewType maskLayout]
            set context [de::open [dm::getCellViews layout -cellName $testcase -libName $destlib] -headless true]
            set design [db::getAttr editDesign -of $context]

            foreach {macro columns rows dx dy x y angle mirror} $floorplans($testcase) {
                switch -- "$angle $mirror" {
                    "0 0"   {set orientation R0}
                    "0 1"   {set orientation MX}
                    "180 1" {set orientation MY}
                    "180 0" {set orientation R180}
                }
                le::createInst -libName $libraries($macro) -cellName $macro -viewName layout -design $design -orient $orientation -origin "[expr $x] [expr $y]" -rows $rows -cols $columns -dx [expr $dx] -dy [expr $dy]
            }
            de::save $context
            de::close $context    
        }
    }

    proc add_boundary_layer{macros halo_size} {
        # macros is a list of macros
        # halo_size: list of 4 numbers
        

        # le::createBoundary <points: [point]> -design <oaDesign> -type <enum> [-force <bool>] [-boundaryLPP <lpp>] [-snapPoints <bool>] [-endShapes <bool>]
        le::createBoundary [list ]
    }


    proc launchCRDTestcaseCreation {} {
        variable scriptDir

        # Source floorplans array from crd_abutment_floorplans.tcl file in same directory as script.
        source [invoke_floorplan $scriptDir]

        # Get list of libraries.
        set libs [db::createList [db::getAttr name -of [dm::getLibs]]]

        # Generate GUI dialog box.
        set crdDialog       [gi::createDialog crdDialog -title "CRD Testcase Creation" -showHelp 0 -execProc cc_crd_abutment::CRDTestcaseCreationExecProc]
        set destlibInput    [dm::createLibInput destlibInput -parent $crdDialog -label "Destination Library"]
        set testcasesInput  [gi::createListInput testcasesInput -parent $crdDialog -label "Testcases to Generate" -header "Testcases" -items [array names floorplans] -showFilter -selectionModel multiple -viewType checkbox]
        set reflibsInput    [gi::createListInput reflibsInput -parent $crdDialog -label "Reference Libraries" -header "Available Selected" -items $libs -selectionModel multiple -viewType dualList]
        set haloInputs      [gi::createInlineGroup haloInputs -parent $crdDialog -label "Boundary Upsize (Halo) (um)"]
        
        foreach side {bottom left right top} {
        set ${side}HaloInput [gi::createNumberInput ${side}HaloInput -parent $haloInputs -label [string toupper $side 0 0] -valueType float -value 5]
        }
        
        set layersInput [gi::createListInput layersInput -parent $crdDialog -label "Layers for Pin Propagation" -header "Layers" -items "M0 M1 M2 M3 M4 M5 M6 M7 M8 M9 M10 M11 M12 M13 M14 M15 M16 M17 M18 MTOP-1 MTOP RDL" -selectionModel multiple -viewType checkbox]
    }



    proc CRDTestcaseCreationExecProc {dialog} {
        
        variable scriptDir [file dirname [info script]]

        # Capture inputs.
        set destlib    [gi::findChild destlibInput.value    -in $dialog]
        set testcases  [gi::findChild testcasesInput.value  -in $dialog]
        set reflibs    [gi::findChild reflibsInput.value    -in $dialog]
        set bottomHalo [gi::findChild bottomHaloInput.value -in $dialog]
        set leftHalo   [gi::findChild leftHaloInput.value   -in $dialog]
        set rightHalo  [gi::findChild rightHaloInput.value  -in $dialog]
        set topHalo    [gi::findChild topHaloInput.value    -in $dialog]
        set layers     [gi::findChild layersInput.value     -in $dialog]
        
        source [invoke_floorplan $scriptDir]

        set macros [get_macros_from_testcases $testcases]
        
        check_lib_layoutview $macros $reflibs

        get_macros_sizes $macros 

        add_boundary_layer $macros [list $bottomHalo $leftHalo $rightHalo $topHalo]

        generate_testcases $testcases $destlib

    }

    launchCRDTestcaseCreation

}

# execute boundary layer? 

le::createBoundary <points: [point]> -design <oaDesign> -type <enum> [-force <bool>] [-boundaryLPP <lpp>] [-snapPoints <bool>] [-endShapes <bool>]

cs::createBoundary [-context <deContext>] [-points [<point>]]

 le::createGuardRing <points: [point]> -design <oaDesign> -master <dmCellView | dmCellViewName | oaDesign> [-params [<map>]] [-evalCallbacks <bool>] [-net <oaNet | string>] 

 cm::createHalo [<name: string>] (-for <{<deFigure>} | {<deSelectionSet>} | {<oaFig>} | {<oaFigGroup>} | {<oaInst>} | {<oaMarker>} | {<oaShape>} | {<oaVia>}> | -in <oaDesign>) [-insts [<string>]] [-left <float>] [-right <float>] [-top <float>] [-bottom <float>]




# pins? 
 Generating Multiple Pins Simultaneously
 
 1. Layout Editor:
    Create > Pin > Generate Pins
	
 2. Generates pins while you are working in a design layout. See Generating Pins Within the Layout.
    Library Manager:
    File > Create Layout Pins
	
Generates pins at the library level. See Generating Layout Pins at the Library Level.

In [ ]:
% % tcl

proc add_boundary_layer {design halo_size} {
    lassign $halo_size halo(x1) halo(y1) halo(x2) halo(y2)
    puts add_boundary_layer

    lassign[db::getAttr bBox -of $design] pointOne pointTwo
    lassign $pointOne point(x1) point(y1)
    lassign $pointOne point(x2) point(y2)

    set new_points[list [list[expr {$point(x1) - $halo(x1)}]
                             [expr {$point(y1) - $halo(y1)}]]
                        [list[expr {$point(x2) + $halo(x2)}]
                        [expr {$point(y2) + $halo(y2)}]]]
    iprint $new_points

    le: createBoundary $new_points - design $design - type pr
    }
}


When to create a new library make the reference to **techlib** 